In [108]:
# a class that represents naire tree
class Node:
    def __init__(self, name, type) -> None:
        self.name = name
        self._children = []
        self._type = type
        self._variabilites = "obligatoire" # facultatif

    def add_child(self, child):
        self._children.append(child)

    def get_children(self):
        return self._children
    
    def get_child(self, index):
        return self._children[index]
    
    def get_child_count(self):
        return len(self._children)
    
    def get_name(self):
        return self.name
    
    def get_type(self):
        return self._type
    
    def is_method(self):
        return self._type == "method"
    
    def is_class(self):
        return self._type == "class"
    
    def __repr__(self) -> str:
        return f"Node({self.name, self._type})({self._children})"
    
    def is_field(self):
        return self._type == "field"

In [109]:
# class that represents an naire tree
class Tree:
    def __init__(self, root) -> None:
        self.root = root

    def __repr__(self) -> str:
        return f"Tree({self.root})"
    
    def get_root(self):
        return self.root
    

In [110]:
ast_1 = {"Account": [("id", "field"), ("balance", "field"), ("Account", "method"), ("deposit", "method"), ("getAmount", "method"), ("withdraw", "method")]}

ast_2 = {"Account": [("id", "field"), ("balance", "field"), ("Account", "method"), ("deposit", "method"), ("getAmount", "method"), ("withdraw", "method"), ("limit", "field"), ("getLimit", "method")]}

ast_3 = {"Account": [("id", "field"), ("balance", "field"), ("Account", "method"), ("deposit", "method"), ("getAmount", "method"), ("withdraw", "method"), ("currency", "field"), ("getCurrency", "method")]}

ast_4 = {"Account": [("id", "field"), ("balance", "field"), ("Account", "method"), ("deposit", "method"), ("getAmount", "method"), ("withdraw", "method"), ("limit", "field"), ("getLimit", "method"), ("currency", "field"), ("getCurrency", "method")]}

ast_as_str = [ast_1, ast_2, ast_3, ast_4]

In [111]:
# convert ast to tree
ast_as_tree = []
for ast in ast_as_str:
    for key, value in ast.items():
        root = Node(key, "class")
        for item in value:
            root.add_child(Node(item[0], item[1]))
        ast_as_tree.append(Tree(root))

print(ast_as_tree[0])




Tree(Node(('Account', 'class'))([Node(('id', 'field'))([]), Node(('balance', 'field'))([]), Node(('Account', 'method'))([]), Node(('deposit', 'method'))([]), Node(('getAmount', 'method'))([]), Node(('withdraw', 'method'))([])]))


In [112]:
def super_sequence_naive(s1, s2):
    # union of two lists
    d = {}
    for node in s1:
        d[node.get_name()] = node
    for node in s2:
        d[node.get_name()] = node
    union = list(d.values())
    return union

    
def super_sequence(list1, list2):
    return super_sequence_naive(list1, list2)
    


# now we will write a function to merge two trees
def merge_trees(tree1, tree2):
    root1 = tree1.get_root()
    root2 = tree2.get_root()
    if root1.name == root2.name:
        children1 = root1.get_children()
        children2 = root2.get_children()
        # fusionne les enfants
        new_children = super_sequence(children1, children2)
        new_root = Node(root1.name, root1.get_type())
        for child in new_children:
            new_root.add_child(child)
        return Tree(new_root)
    else:
        return [tree1, tree2]

In [113]:
# now we will write a function to merge a list of trees
def merge_list_of_trees(trees):
    new_trees = []
    for i in range(len(trees)):
        for j in range(i + 1, len(trees)):
            new_trees.append(merge_trees(trees[i], trees[j]))
    return new_trees

In [114]:
ast_as_tree1 = ast_as_tree.copy()
t0 = ast_as_tree1[0]
t1 = ast_as_tree1[1]
t = merge_trees(t0, t1)
print(t.get_root().get_child_count())
print(t)

8
Tree(Node(('Account', 'class'))([Node(('id', 'field'))([]), Node(('balance', 'field'))([]), Node(('Account', 'method'))([]), Node(('deposit', 'method'))([]), Node(('getAmount', 'method'))([]), Node(('withdraw', 'method'))([]), Node(('limit', 'field'))([]), Node(('getLimit', 'method'))([])]))
